In [59]:
import pandas as pd
from datetime import timedelta

In [60]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

## 4) Devise a method of generating a table of all Accounts with their “nominal” expected payments as of any arbitrary timestamp in the past. In other words, we’re looking for the amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.

In [ ]:
# groups.head()

In [61]:
# Create new column with the number of payments needed to pay the 'price_unlock' amount if the minimum payment was made each time.
groups['max_num_payments'] = (groups['price_unlock'] - groups['price_upfront']) / groups['minimum_payment']
# groups.head()

### amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.

* merge groups_df w/ ast_df (may need to look at payments_df?)
* merge several: ast account_id to find product_id
* Count number of times each account has gone into disabled
* Count of times disabed TIMES(*) min payment + down payment

## Merge Dataframes

In [62]:
# Merge 'ast' and 'accounts' dataframes
merged_df = pd.merge(ast, accounts, how='outer')

# Limit columns
merged_df = merged_df[['started_when','account_id','to_state','group_id','is_written_off','is_unlocked']]
merged_df.head()

,started_when,account_id,to_state,group_id,is_written_off,is_unlocked
0,2019-11-06 05:49:39.571392,2.0,ENABLED,1.0,False,False
1,2019-11-25 22:20:59.150339,2.0,DISABLED,1.0,False,True
2,2019-11-26 10:22:36.571392,2.0,ENABLED,1.0,False,False
3,2019-11-29 11:03:32.150339,2.0,DISABLED,1.0,False,True
4,2019-12-03 10:08:03.571392,2.0,ENABLED,1.0,False,False


In [63]:
# Merge groups dataframe
merged_df = pd.merge(left=merged_df, right=groups, how='outer', left_on='group_id', right_on='id')

# Limit columns
merged_df = merged_df.drop(columns=['id','name','price_clock_hour'])
merged_df.head()

,started_when,account_id,to_state,group_id,is_written_off,is_unlocked,price_upfront,price_unlock,minimum_payment,max_num_payments
0,2019-11-06 05:49:39.571392,2.0,ENABLED,1.0,False,False,950.0,8100.0,71.5,100.0
1,2019-11-25 22:20:59.150339,2.0,DISABLED,1.0,False,True,950.0,8100.0,71.5,100.0
2,2019-11-26 10:22:36.571392,2.0,ENABLED,1.0,False,False,950.0,8100.0,71.5,100.0
3,2019-11-29 11:03:32.150339,2.0,DISABLED,1.0,False,True,950.0,8100.0,71.5,100.0
4,2019-12-03 10:08:03.571392,2.0,ENABLED,1.0,False,False,950.0,8100.0,71.5,100.0


## Limit by Indicated Timestamp

In [ ]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

In [ ]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

# Adjust timestamp to end of day
timestamp_dt = timestamp_dt + timedelta(hours=23,minutes=59,seconds=59)

In [ ]:
# Change data type from string to datetime object
merged_df['started_when'] = pd.to_datetime(merged_df['started_when'])

In [ ]:
# Create limited dataframe based on indicated timestamp 
merged_limited_df = merged_df.loc[merged_df['started_when'] <= timestamp_dt]

In [ ]:
# Accout_ID 1 = Group_ID 1 = price_upfront 950 = price_unlock 8100, min_payment 71.5
merged_limited_df.sort_values('account_id', ascending=True)

In [ ]:
merged_limited_df.account_id.value_counts()